In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hii'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-trace-misc

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
## You will need to run jupyter from that directory.
## source $TEMP_BIN/_dir-aliases.bash
source _dir-aliases.bash
alias | wc -l

8


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1210)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-1210

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-trace-misc/test-1210


In [6]:
# Count aliases proper
alias | wc -l

9


In [7]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

2


In [8]:
##1 SHOWS HISTORY OF BASH COMMANDS
alias hist='history $LINES'
hist

 2029  source _dir-aliases.bash
 2030  alias | wc -l
 2031  echo $?
 2032  ## NOTE: For reproducability, the directory name needs to be fixed
 2033  ## In place of $$, use a psuedo random number (e,g., 1210)
 2034  ## *** All output from one run to the next needs to be the same ***
 2035  ## temp_dir=$TMP/test-$$
 2036  temp_dir=$TMP/test-1210
 2037  mkdir -p "$temp_dir"
 2038  # TODO: /bin/rm -rvf "$temp_dir"
 2039  cd "$temp_dir"
 2040  pwd
 2041  #ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
 2042  alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"
 2043  echo $?
 2044  # Count aliases proper
 2045  alias | wc -l
 2046  echo $?
 2047  # Count functions
 2048  typeset -f | egrep '^\w+' | wc -l
 2049  echo $?
 2050  ##1 SHOWS HISTORY OF BASH COMMANDS
 2051  alias hist='history $LINES'
 2052  hist


In [9]:
##2 SHOWS HISTORY WITHOUT TIMESTAMPS
# Removes timestamp from history (e.g., " 1972  [2014-05-02 14:34:12] dir *py" => " 1972  dir *py")
# TEST: function hist { h | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }
# note: funciton used to simplify specification of quotes

# quiet-unalias h #COMMAND NOT FOUND
function h { hist | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }

h

 2037  mkdir -p "$temp_dir"
 2038  # TODO: /bin/rm -rvf "$temp_dir"
 2039  cd "$temp_dir"
 2040  pwd
 2041  #ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
 2042  alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"
 2043  echo $?
 2044  # Count aliases proper
 2045  alias | wc -l
 2046  echo $?
 2047  # Count functions
 2048  typeset -f | egrep '^\w+' | wc -l
 2049  echo $?
 2050  ##1 SHOWS HISTORY OF BASH COMMANDS
 2051  alias hist='history $LINES'
 2052  hist
 2053  echo $?
 2054  ##2 SHOWS HISTORY WITHOUT TIMESTAMPS
 2055  # Removes timestamp from history (e.g., " 1972  [2014-05-02 14:34:12] dir *py" => " 1972  dir *py")
 2056  # TEST: function hist { h | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }
 2057  # note: funciton used to simplify specification of quotes
 2058  # quiet-unalias h #COMMAND NOT FOUND
 2059  function h { hist | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }
 2060  h


In [10]:
##3 ASCTIME
# EX: $ asctime | perl -pe 's/\d/N/g; s/\w+ \w+/DDD MMM/;' => "DDD MMM NN NN:NN:NN NNNN"
function asctime() { perl -e "print (scalar localtime($1));"; echo ""; }

asctime | perl -pe 's/\d/N/g; s/\w+ \w+/DDD MMM/;'

DDD MMM  N NN:NN:NN NNNN


In [11]:
##4 FILTER DIRECTORY NAMES
# filter-dirnames: strip directory names from ps listing (TODO: rename as strip-dirnames)
function filter-dirnames () { perl -pe 's/\/[^ \"]+\/([^ \/\"]+)/$1/g;'; }

####TEST LEFT####

In [12]:
##5 COMMAIZE NUMBERS
# comma-ize-number(): add commas to numbers in stdin
# EX: echo "1234567890" | comma-ize-number => 1,234,567,890
function comma-ize-number () { perl -pe 'while (/\d\d\d\d/) { s/(\d)(\d\d\d)([^\d])/\1,\2\3/g; } '; }

echo "1012342305324254" | comma-ize-number

1,012,342,305,324,254


In [15]:
##6 APPLYING NUMERIC SUFFIXES

# apply-numeric-suffixes([once=0]): converts numbers in stdin to use K/M/G suffixes.
# Notes:
# - K, M, G and T based on powers of 1024.
# - If $once non-zero, then the substitution is only applied one-time per line.
# - The number must be preceded by a word boundary and followed by whitespace.
# This was added in support of the usage function (e.g., numeric subdirectory names).
# TODO:
# - Convert to Perl script to avoid awkward bash command line construction.
# - Make the trailing context a word-boundry as well (rather than whitespace).
# EX: echo "1024 1572864 1073741824" | apply-numeric-suffixes => 1K 1.5M 1G
# EX: echo "1024 1572864 1073741824" | apply-numeric-suffixes 1 => 1K 1572864 1073741824
function apply-numeric-suffixes () {
    local just_once="$1"
    local g="g";
    if [ "$just_once" = "1" ]; then g=""; fi
    # TODO: make only sure the first number is converted if just-once applies
    # NOTE: 3 args to sprintf: coefficient, KMGT suffix, and post-context
    ## DEBUG; perl -pe '$suffixes="_KMGT";  s@\b(\d{4,15})(\s)@$pow = log($1)/log(1024);  $new_num=($1/1024**$pow);  $suffix=substr($suffixes, $pow, 1);  print STDERR ("s=$suffixes p=$pow nn=$new_num l=$suffix\n"); sprintf("%.3g%s%s", $new_num, $suffix, $2)@e'"$g;"
    perl -pe '$suffixes="_KMGT";  s@\b(\d{4,15})(\s)@$pow = int(log($1)/log(1024));  $new_num=($1/1024**$pow);  $suffix=substr($suffixes, $pow, 1);  sprintf("%.3g%s%s", $new_num, $suffix, $2)@e'"$g;"
}

# TESTED COMMAND (if 1 KB ~ 1000 B; must return 7.XX GB (~ 8 GB) )
echo "8000000000" | apply-numeric-suffixes

7.45G


In [17]:
##6.1 APPLYING USAGE NUMERIC SUFFIXES
# apply-usage-numeric-suffixes(): factors in 1k blocksize before applying numeric suffixes
# note: (?=\s) is lookahead pattern (see perlre manpage)
#
function apply-usage-numeric-suffixes() {
    perl -pe 's@^(\d+)(?=\s)@$1 * 1024@e;' | apply-numeric-suffixes 1
}

# IN SHORT = result * 1000
echo "8000000000" | apply-usage-numeric-suffixes #G -> T
echo "8000000" | apply-usage-numeric-suffixes #M -> G

7.45T
7.63G
